# Astro Vibe Bot — Local Test Notebook

Use this notebook to generate a daily context, preview the channel message for all 12 signs, and optionally send it to a Telegram chat or channel.

Prereqs:
- `.env` must exist with `TELEGRAM_BOT_TOKEN` and `OPENAI_API_KEY`
- Optional: set `RSS_FEED_URL`, `TIMEZONE`, and `TEST_CHAT_ID` in `.env`


In [1]:
import os
import asyncio
from pathlib import Path
from zoneinfo import ZoneInfo

from dotenv import load_dotenv
from openai import OpenAI
from aiogram import Bot

from main import load_signs, generate_daily_context, build_channel_sign_messages

root_dir = Path.cwd()
if not (root_dir / ".env").exists() and (root_dir.parent / ".env").exists():
    root_dir = root_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RSS_FEED_URL = os.getenv("RSS_FEED_URL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TEST_CHAT_ID = os.getenv("TEST_CHAT_ID")
TIMEZONE = os.getenv("TIMEZONE", "UTC")
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
TELEGRAM_NEWS_CHANNEL = os.getenv("TELEGRAM_NEWS_CHANNEL")
TELEGRAM_NEWS_LIMIT = int(os.getenv("TELEGRAM_NEWS_LIMIT", "20"))
TELETHON_SESSION = os.getenv("TELETHON_SESSION", "telethon.session")
TELETHON_SESSION_STRING = os.getenv("TELETHON_SESSION_STRING")

if not TELEGRAM_BOT_TOKEN or not OPENAI_API_KEY:
    raise RuntimeError("Set TELEGRAM_BOT_TOKEN and OPENAI_API_KEY in .env")

client = OpenAI(api_key=OPENAI_API_KEY)
signs = load_signs()
timezone = ZoneInfo(TIMEZONE)
telegram_source = {
    "api_id": int(TELEGRAM_API_ID) if TELEGRAM_API_ID else None,
    "api_hash": TELEGRAM_API_HASH,
    "channel": TELEGRAM_NEWS_CHANNEL,
    "limit": TELEGRAM_NEWS_LIMIT,
    "session_path": TELETHON_SESSION,
    "session_string": TELETHON_SESSION_STRING,
}


In [2]:
# Generate today's global context
context = await generate_daily_context(
    client,
    signs,
    RSS_FEED_URL,
    OPENAI_MODEL,
    telegram_source=telegram_source,
)
context["global_summary"], list(context["vibes"].keys())[:3]


[debug] Telegram news context:
- Тепер ціна на поїзд буде залежати від попиту, а повернення квитків відбуватиметься за новими правилами, — «РБК-Україна» цитує «УЗ» НОВИНА | Підписатися
- LELÉKA представлять Україну на «Євробаченні-2026» НОВИНА | Підписатися
- ❗️Працівники ТЦК забили до смерті 55-річного чоловіка, — поліція Дніпропетровщини Інформація про смерть чоловіка надійшла сьогодні вночі. Трьох працівників ТЦК затримано. В авто виявили сліди крові. НОВИНА | Підписатися
- Кадри удару по ТЕЦ «Луч» пакетом GMLRS РСЗВ HIMARS сьогодні зранку НОВИНА | Підписатися
- ⚡️ Мусимо пройти найближчі кілька діб, що будуть дуже складними для Києва. У столиці знову прогнозують сильні морози, зокрема, вночі, — Кличко НОВИНА | Підписатися
- В людей шок! У Буковелі почали відключати світло На відео момент, коли курорт переходив на генератори. НОВИНА | Підписатися
- У російському бєлгороді масовий блекаут Губернатор області вже бідкається, що БпЛА атакували місцеву ТЕЦ "Луч" та електропідстанцію "бєл

('Складні часи потребують витривалості та підтримки одне одного.',
 ['Aries', 'Taurus', 'Gemini'])

In [3]:
# Build the daily channel messages (12 separate posts)
channel_messages = build_channel_sign_messages(context, signs)
channel_messages


['Сьогодні ви здатні подолати будь-які труднощі!\nСкладні часи потребують витривалості та підтримки одне одного.\n\n♈ Овен: Цей день вимагає від вас сміливості та рішучості. Ваша енергія зараз на висоті, використовуйте її для досягнення цілей. У стосунках проявіть свою відданість, адже ваша пристрасть зараз вражає. Грошові питання можуть бути нестабільними, але ви знайдете спосіб вирішити їх.',
 '♉ Телець: Сьогодні варто зосередитися на стабільності та спокої. Ваш практичний підхід допоможе вам впоратися з будь-якими викликами. У коханні ваша обережність принесе гармонію та спокій. У фінансах зберігайте обережність, не ризикуйте зайвий раз.',
 "♊ Близнюки: Сьогодні ви можете виявити свою допитливість у нових проектах. Ваші ідеї можуть стати основою для успіху. У стосунках будьте відкритими до спілкування, це зміцнить зв'язки. Фінансові можливості можуть з'явитися несподівано, тож будьте готові до змін.",
 '♋ Рак: Приділіть увагу своїм емоціям і потребам близьких сьогодні. Ваша чутливіс

In [4]:
# Optional: send the daily channel messages
# Set TEST_CHAT_ID in .env (example: your numeric chat_id or @TarotReaderOracle)

if TEST_CHAT_ID:
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    for message in channel_messages:
        await bot.send_message(TEST_CHAT_ID, message)
    await bot.session.close()
    "Sent"
else:
    "TEST_CHAT_ID not set"
